## Import

In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch
!pip install sentencepiece
!pip install gluonnlp==0.9.1
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
!wget https://www.dropbox.com/s/374ftkec978br3d/ratings_train.txt?dl=1
!wget https://www.dropbox.com/s/977gbwh542gdy94/ratings_test.txt?dl=1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 17.6 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 kB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp310-cp310-linux_x86_64.whl size=681081 sha256=396eb47ff9eb8263673d0d64ef28ad66a7db1397471acb1d4509722a39f16ccb
  Stored in directory: /root/.cache/pip/wheels/1a/1e/0d/99f55911d90f2b95b9f7c176d5813ef3622894a4b30fde6bd3
Successfully built gluonnlp
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 91.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 72.7 MB/s eta 0:00:00
     ━━━━━━━━

In [ ]:
import torch
from torch import nn
import pandas as pd
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
from sklearn.metrics import classification_report, f1_score
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import recall_score, classification_report, auc, roc_curve
from sklearn.metrics import precision_recall_fscore_support, f1_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_recall_fscore_support,f1_score,accuracy_score


from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
#GPU 사용
device = torch.device("cuda:0")
#CPU
#device = torch.device('cpu')

In [ ]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
model = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


## Class Definition



## BERTSentenceTransform
* tokenize<br>
*  padding



In [ ]:
class BERTSentenceTransform:

    def __init__(self, tokenizer, max_seq_length,vocab, pad=True, pair=True):
        self._tokenizer = tokenizer
        self._max_seq_length = max_seq_length
        self._pad = pad
        self._pair = pair
        self._vocab = vocab

    def __call__(self, line):

        # convert to unicode
        text_a = line[0]
        if self._pair:
            assert len(line) == 2
            text_b = line[1]

        tokens_a = self._tokenizer.tokenize(text_a)
        tokens_b = None

        if self._pair:
            tokens_b = self._tokenizer(text_b)

        if tokens_b:
            self._truncate_seq_pair(tokens_a, tokens_b,
                                    self._max_seq_length - 3)
        else:

            if len(tokens_a) > (self._max_seq_length - 2):
                tokens_a = tokens_a[0:(self._max_seq_length - 2)]


        vocab = self._vocab
        tokens = []
        tokens.append(vocab.cls_token)
        tokens.extend(tokens_a)
        tokens.append(vocab.sep_token)
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens.extend(tokens_b)
            tokens.append(vocab.sep_token)
            segment_ids.extend([1] * (len(tokens) - len(segment_ids)))

        input_ids = self._tokenizer.convert_tokens_to_ids(tokens)

        # The valid length of sentences. Only real  tokens are attended to.
        valid_length = len(input_ids)

        # padding 패딩
        if self._pad:
            # Zero-pad up to the sequence length.
            padding_length = self._max_seq_length - valid_length
            # use padding tokens for the rest
            input_ids.extend([vocab[vocab.padding_token]] * padding_length)
            segment_ids.extend([0] * padding_length)

        return np.array(input_ids, dtype='int32'), np.array(valid_length, dtype='int32'),\
            np.array(segment_ids, dtype='int32')


##BERTDataset
* BERTSentenceTransform(Dataset)

In [ ]:
class BERTDataset(Dataset):
  def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len, pad, pair):
    transform = BERTSentenceTransform(
        bert_tokenizer, max_seq_length=max_len, vocab=vocab, pad=pad,pair=pair)

    self.sentences = [transform([i[sent_idx]]) for i in dataset]
    self.labels = [np.int32(i[label_idx]) for i in dataset]

  def __getitem__(self, i):
    return (self.sentences[i] + (self.labels[i], ))

  def __len__(self):
    return (len(self.labels))

##BERTClassifier
* accuracy_score
* setting parameters
* fit
* predict
* attention mask
* forward

In [ ]:
class BERTClassifier(nn.Module):
  def __init__(self,
               bert,
               hidden_size=768,
               num_classes=2,
               dr_rate=None,
               params=None):
    super(BERTClassifier, self).__init__()
    self.bert = bert
    self.dr_rate = dr_rate
    self.num_classes = num_classes

    self.classifier = nn.Linear(hidden_size , num_classes)
    if dr_rate:
      self.dropout = nn.Dropout(p=dr_rate)
  def score(self, X, y):
        # 모델의 예측값 계산
        predictions = self.predict(X)
        # 스코어링 지표 계산
        score = accuracy_score(y, predictions)
        return score
  def set_params(self, **params):
    if 'dr_rate' in params:
      self.dr_rate = params['dr_rate']
    if 'hidden_size' in params:
      self.classifier = nn.Linear(params['hidden_size'], self.num_classes)
  def fit(self, X, y):
    pass
  def predict(self, X):
    with torch.no_grad():
      outputs = self.bert(X)
      logits = outputs[0]
    predictions = torch.softmax(logits, dim=1)
    return predictions.numpy()

  # attention mask 어텐션 마스크
  def gen_attention_mask(self, token_ids, valid_length):
    attention_mask = torch.zeros_like(token_ids)
    for i, v in enumerate(valid_length):
      attention_mask[i][:v] = 1
    return attention_mask.float()

  def forward(self, token_ids, valid_length, segment_ids):
    attention_mask = self.gen_attention_mask(token_ids, valid_length)

    _, pooler = self.bert(input_ids=token_ids, token_type_ids=segment_ids.long(), attention_mask=attention_mask.float().to(token_ids.device))
    if self.dr_rate:
        pooler = self.dropout(pooler)
    out = self.classifier(pooler)
    return out

## Data Load

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path = '/content/drive/MyDrive/광고성기사/news_preprocess.csv'
df = pd.read_csv(path,index_col=0, encoding='cp949');

In [ ]:
import torch.cuda

class MyBERTClassifier:
    def __init__(self, tokenizer, bertmodel, vocab, max_len=71, batch_size=64, warmup_ratio=0.1,
                 num_epochs=5, max_grad_norm=1, log_interval=200, learning_rate=5e-5,
                 params=None, device='cpu'):
        self.tokenizer = tokenizer
        self.bertmodel = bertmodel
        self.vocab = vocab
        self.max_len = max_len
        self.batch_size = batch_size
        self.warmup_ratio = warmup_ratio
        self.num_epochs = num_epochs
        self.max_grad_norm = max_grad_norm
        self.log_interval = log_interval
        self.learning_rate = learning_rate
        self.params = params
        self.device = device
        self.model = BERTClassifier(model)
        self.model.to(self.device)

    def prepare_data(self, df):
        # Shuffle the DataFrame
        df = df.sample(frac=1)
        df.reset_index(drop=True, inplace=True)

        # Split the data
        target = df['label']
        x_train, x_valid, y_train, y_valid = train_test_split(df, target, test_size=0.2, shuffle=True, stratify=target,
                                                              random_state=34)

        train = x_train.loc[:, ['label', 'content']]
        valid = x_valid.loc[:, ['label', 'content']]

        dataset_train = []
        for q, label in zip(train['content'], train['label']):
            data = []
            data.append(q)
            data.append(str(label))
            dataset_train.append(data)

        dataset_valid = []
        for q, label in zip(valid['content'], valid['label']):
            data = []
            data.append(q)
            data.append(str(label))
            dataset_valid.append(data)

        self.data_train = BERTDataset(dataset_train, 0, 1, self.tokenizer, self.vocab, self.max_len, True, False)
        self.data_test = BERTDataset(dataset_valid, 0, 1, self.tokenizer, self.vocab, self.max_len, True, False)

        self.train_dataloader = torch.utils.data.DataLoader(self.data_train, batch_size=self.batch_size, num_workers=5)
        self.test_dataloader = torch.utils.data.DataLoader(self.data_test, batch_size=self.batch_size, num_workers=5)

    def train(self, loss_fn=nn.CrossEntropyLoss()):
        t_total = len(self.train_dataloader) * self.num_epochs
        warmup_step = int(t_total * self.warmup_ratio)

        optimizer = AdamW(self.model.parameters(), lr=self.learning_rate, weight_decay=0.01)
        scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step,
                                                    num_training_steps=t_total)

        best_epoch = 0
        best_valid_acc = 0.0

        for e in range(self.num_epochs):
            train_acc = 0.0
            test_acc = 0.0
            self.model.train()
            train_predictions = []

            for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(self.train_dataloader)):
                optimizer.zero_grad()
                token_ids = token_ids.long().to(self.device)
                segment_ids = segment_ids.long().to(self.device)
                valid_length = valid_length
                label = label.long().to(self.device)
                out = self.model(token_ids, valid_length, segment_ids)
                loss = loss_fn(out, label)
                loss.backward()
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.max_grad_norm)
                optimizer.step()
                scheduler.step()
                train_acc += self.calc_accuracy(out, label)
                predictions = torch.argmax(out, dim=1)
                train_predictions.extend(predictions.cpu().numpy())
                if batch_id % self.log_interval == 0:
                    print("epoch {} batch id {} loss {} train acc {}".format(e + 1, batch_id + 1,
                                                                              loss.data.cpu().numpy(),
                                                                              train_acc / (batch_id + 1)))
            print("epoch {} train acc {}".format(e + 1, train_acc / (batch_id + 1)))

            valid_predictions = []
            self.model.eval()
            for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(self.test_dataloader)):
                token_ids = token_ids.long().to(self.device)
                segment_ids = segment_ids.long().to(self.device)
                valid_length = valid_length
                label = label.long().to(self.device)
                out = self.model(token_ids, valid_length, segment_ids)
                test_acc += self.calc_accuracy(out, label)
                predictions = torch.argmax(out, dim=1)
                valid_predictions.extend(predictions.cpu().numpy())
                valid_acc = test_acc / (batch_id + 1)
                if valid_acc > best_valid_acc:
                    best_valid_acc = valid_acc
                    best_epoch = e

            print("epoch {} valid acc {}".format(e + 1, test_acc / (batch_id + 1)))

        print(f"Best Epoch: {best_epoch + 1}")
        print(f"Best Validation Accuracy: {best_valid_acc}")

        # Classification report
        valid['pred'] = valid_predictions
        train['pred'] = train_predictions
        valid = pd.DataFrame(valid)
        train = pd.DataFrame(train)
        origin = list(valid['label'])
        pred = list(valid_predictions)
        print(classification_report(origin, pred))

    def calc_accuracy(self, X, Y):
        max_vals, max_indices = torch.max(X, 1)
        train_acc = (
                max_indices == Y
        ).sum().data.cpu().numpy() / max_indices.size()[0]
        return train_acc

    def predict(self, sentence):
        dataset = [[sentence, '0']]
        test = BERTDataset(dataset, 0, 1, self.tokenizer, self.vocab, self.max_len, True, False)
        test_dataloader = torch.utils.data.DataLoader(
            test, batch_size=self.batch_size, num_workers=2
        )
        self.bertmodel.to(self.device)  # Move the model to GPU
        answer = 0
        for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
            token_ids = token_ids.long().to(self.device)
            segment_ids = segment_ids.long().to(self.device)
            valid_length = valid_length
            label = label.long().to(self.device)
            out = self.bertmodel(token_ids, valid_length, segment_ids)
            for logits in out:
                logits = logits.detach().cpu().numpy()
                answer = np.argmax(logits)

    #def grid_search(self, train_texts, train_labels, param_grid, cv=5):
        # Convert train_texts to a flat list of strings
     #   train_texts_flat = [' '.join(texts) for texts in train_texts]

        # Define the pipeline
      #  pipeline = Pipeline([
       #     ('vectorizer', TfidfVectorizer()),
        #    ('classifier', self.model),
        #])

        # Perform grid search
        #grid_search = GridSearchCV(pipeline, param_grid, cv=cv)
        #grid_search.fit(train_texts_flat, train_labels)

        # Print the best parameters and performance
        #print("Best Parameters: ", grid_search.best_params_)

## Train Model
#### Optimizer : AdamW<br>Scheduler

In [ ]:
# Instantiate the MyBERTClassifier
classifier = MyBERTClassifier(tokenizer, model,vocab)

# Prepare the data
classifier.prepare_data(df)

# Train the classifier
classifier.train()

# Perform grid search
#param_grid = {
#    'classifier__dr_rate': [0.2, 0.5],
#    'classifier__hidden_size': [256, 512],
#   'classifier__num_classes': [2, 3]
#}
#classifier.grid_search(classifier.data_train.sentences, classifier.data_train.labels, param_grid, cv=5)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:560: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
<ipython-input-10-638c6552195f>:72: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segme

  0%|          | 0/379 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 0.7347166538238525 train acc 0.40625


In [ ]:
df=pd.concat([train,valid])
df.to_csv('/content/drive/MyDrive/news_pred.csv')